In [1]:
import pandas as pd
import os
import torch
import librosa
import librosa.display
import pywt
import csv
import pickle
import keras

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import *
from keras.callbacks import EarlyStopping

In [2]:
FOLDER_PATH = './dataset/fr/clips/'
FEATURES_FILE = 'matthieu_features.pkl'

In [3]:
def padding(data, axis, length):
    pad_width = [(0, 0)] * len(data.shape)
    pad_width[axis] = (0, max(0, length - data.shape[axis]))
    padded_data = np.pad(data, pad_width, mode='constant', constant_values=0)
    return padded_data

def get_features(df_in, max_len):
    features = []
    labels = []

    for index in range(0, len(df_in)):
        filename = FOLDER_PATH + df_in.path[index]
        label = df_in.sentence[index]

        # load the file
        y, sr = librosa.load(filename)
        
        # Trim the audio file to remove leading/trailing silence
        y, _ = librosa.effects.trim(y)

        # Compute MFCC
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # n_mfcc: nombre de coefficients MFCC à conserver
        
        print(mfccs.shape)
        
        # Pad/truncate MFCCs
        if (mfccs.shape[1] < max_len):
            pad_width = max_len - mfccs.shape[1]
            mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        else:
            mfccs = mfccs[:, :max_len]
        
        features.append(mfccs.T)
        labels.append(label)

    return (features, labels)

def preprocess_data(X, y):
    print(np.mean(X))
    print(np.std(X))
    
    X = np.array((X - np.mean(X)) / np.std(X))
    y = np.array(y)
    return X, y

def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=123, stratify=y)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=123)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [4]:
data = pd.read_csv('./dataset/fr/test.tsv', delimiter='\t')
data = data[['path', 'sentence']]

In [5]:
max_len = 200

if os.path.exists(FEATURES_FILE):
    with open(FEATURES_FILE, 'rb') as f:
        X, y = pickle.load(f)
else:
    X, y = get_features(data, max_len)
    with open(FEATURES_FILE, 'wb') as f:
        pickle.dump((X, y), f)

(13, 135)
(13, 85)
(13, 65)
(13, 61)
(13, 63)
(13, 90)
(13, 234)
(13, 111)
(13, 204)
(13, 121)
(13, 139)
(13, 234)
(13, 117)
(13, 112)
(13, 60)
(13, 139)
(13, 143)
(13, 90)
(13, 92)
(13, 90)
(13, 102)
(13, 85)
(13, 128)
(13, 87)
(13, 157)
(13, 113)
(13, 81)
(13, 116)
(13, 159)
(13, 134)
(13, 107)
(13, 90)
(13, 110)
(13, 104)
(13, 259)
(13, 218)
(13, 43)
(13, 143)
(13, 85)
(13, 99)
(13, 72)
(13, 181)
(13, 110)
(13, 65)
(13, 267)
(13, 68)
(13, 84)
(13, 82)
(13, 97)
(13, 137)
(13, 154)
(13, 139)
(13, 100)
(13, 127)
(13, 23)
(13, 119)
(13, 102)
(13, 327)
(13, 91)
(13, 105)
(13, 117)
(13, 62)
(13, 131)
(13, 135)
(13, 172)
(13, 135)
(13, 102)
(13, 84)
(13, 88)
(13, 93)
(13, 67)
(13, 135)
(13, 110)
(13, 89)
(13, 139)
(13, 74)
(13, 76)
(13, 113)
(13, 114)
(13, 97)
(13, 71)
(13, 87)
(13, 90)
(13, 201)
(13, 148)
(13, 108)
(13, 90)
(13, 88)
(13, 99)
(13, 108)
(13, 138)
(13, 100)
(13, 117)
(13, 233)
(13, 257)
(13, 148)
(13, 55)
(13, 129)
(13, 150)
(13, 72)
(13, 103)
(13, 77)
(13, 56)
(13, 46)
(13,

In [6]:
X, y = preprocess_data(X, y)

-17.961946
107.19872


In [7]:
def create_model(input_shape):
    model = keras.Sequential()

    # Couche Conv1D
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))

    # Couche LSTM pour traiter chaque trame
    model.add(LSTM(64, return_sequences=True))
    model.add(LSTM(32))

    # Couche Dense
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    # Couche de sortie
    model.add(Dense(4, activation='softmax'))

    # Choix de l'optimiseur avec un taux d'apprentissage adapté
    opt = keras.optimizers.Adam(learning_rate=0.01)

    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

    return model


In [8]:
# Divisez les données
X_train, X_test, X_val, y_train, y_test, y_val = split_data(X, y)

# Maintenant, supposons que X a la forme (nombre_d'échantillons, nombre_de_trames, n_mfcc)
num_samples, num_frames, n_mfcc = X_train.shape
input_shape = (num_frames, n_mfcc)

# Convertir les étiquettes en format catégorique
num_classes = 4
y_train = to_categorical(y_train - 1, num_classes)
y_val = to_categorical(y_val - 1, num_classes)
y_test = to_categorical(y_test - 1, num_classes)

# Créez le modèle
model = create_model(input_shape)
model.summary()

# Ajoutez le critère d'arrêt précoce à la liste des callbacks pour l'entraînement
history = model.fit(X_train, y_train, epochs=400, batch_size=512, validation_data=(X_val, y_val))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 198, 64)           2560      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 99, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 99, 64)            0         
                                                                 
 lstm (LSTM)                 (None, 99, 64)            33024     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense (Dense)               (None, 256)               8448      
                                                        

KeyboardInterrupt: 

In [23]:
# Évaluez le modèle
TrainLoss, Trainacc = model.evaluate(X_train, y_train)
TestLoss, Testacc = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

13/13 [==============================] - 2s 51ms/step


In [24]:
print('Confusion_matrix: ', tf.math.confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

# Sauvegardez le modèle
model.save("model.h5")

Confusion_matrix:  tf.Tensor(
[[ 85   1   3   5]
 [  3 100   1   5]
 [  3   0  99   2]
 [  5   0   1  96]], shape=(4, 4), dtype=int32)
